In [ ]:
!pip install pytorch-lightning pytorch-forecasting torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 46.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Favorita_Subset/'

Mounted at /content/drive


In [ ]:
# path = './Favorita_Subset/'

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import RMSE
from torch.utils.data import DataLoader
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping
import torch

In [ ]:
load_train = torch.load(path + "train_mini.pth")
load_val = torch.load(path + "val_mini.pth")

<ipython-input-16-e22ffdf3d5ed>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_train = torch.load(path + "train_mini.pth")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Favorita_Subset/train_mini.pth'

In [ ]:
print(load_train)

TimeSeriesDataSet[length=16383003](
	time_idx='time_idx',
	target='unit_sales',
	group_ids=['store_nbr', 'family', 'class'],
	weight=None,
	max_encoder_length=30,
	min_encoder_length=15,
	min_prediction_idx=0,
	min_prediction_length=7,
	max_prediction_length=7,
	static_categoricals=['store_nbr', 'family', 'store_type', 'item_nbr', 'perishable', 'class', 'cluster'],
	static_reals=None,
	time_varying_known_categoricals=['weekday', 'holiday_description', 'holiday_type', 'onpromotion'],
	time_varying_known_reals=['dcoilwtico', 'Min Temperature (C)', 'Max Temperature (C)'],
	time_varying_unknown_categoricals=None,
	time_varying_unknown_reals=None,
	variable_groups=None,
	constant_fill_strategy=None,
	allow_missing_timesteps=True,
	lags=None,
	add_relative_time_idx=True,
	add_target_scales=False,
	add_encoder_length=True,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation=None,
	method_kwargs={}
),
	categorical_encoders={'__group_id__sto

In [ ]:
print(load_train.categorical_encoders)

None


In [ ]:
train_loader = TimeSeriesDataSet.to_dataloader(load_train,batch_size=64,shuffle=True)
val_loader = TimeSeriesDataSet.to_dataloader(load_val,batch_size=64,shuffle=False)

In [ ]:
tft = TemporalFusionTransformer.from_dataset(load_train)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/models/temporal_fusion_transformer/__init__.py:171: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is installed, in order to minimize the number of 

In [ ]:
print(tft)

TemporalFusionTransformer(
  	"attention_head_size":               4
  	"categorical_groups":                {}
  	"causal_attention":                  True
  	"dataset_parameters":                {'time_idx': 'time_idx', 'target': 'unit_sales', 'group_ids': ['store_nbr', 'family', 'class'], 'weight': None, 'max_encoder_length': 30, 'min_encoder_length': 15, 'min_prediction_idx': 0, 'min_prediction_length': 7, 'max_prediction_length': 7, 'static_categoricals': ['store_nbr', 'family', 'store_type', 'item_nbr', 'perishable', 'class', 'cluster'], 'static_reals': None, 'time_varying_known_categoricals': ['weekday', 'holiday_description', 'holiday_type', 'onpromotion'], 'time_varying_known_reals': ['dcoilwtico', 'Min Temperature (C)', 'Max Temperature (C)'], 'time_varying_unknown_categoricals': None, 'time_varying_unknown_reals': None, 'variable_groups': None, 'constant_fill_strategy': None, 'allow_missing_timesteps': True, 'lags': None, 'add_relative_time_idx': True, 'add_target_scales': F

In [ ]:
print(torch.cuda.is_available())

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint

# Define a ModelCheckpoint callback to save the best model based on validation loss
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Metric to monitor
    dirpath="checkpoints/",  # Directory to save the model checkpoints
    filename="best_model-{epoch:02d}-{val_loss:.2f}",  # Filename pattern for saved models
    save_top_k=1,  # Save only the best model
    mode="min",  # Minimizing validation loss
)

trainer = Trainer(
    max_epochs=3,
    gradient_clip_val=0.1,
    limit_train_batches=1.0,  # Use the full training dataset
    callbacks=[checkpoint_callback],  # Add the checkpoint callback
)

trainer.fit(tft, train_loader, val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 59.9 K | train
3  | prescalers                         | ModuleDict                      | 80     | train
4  | static_variable_selection          | VariableSelectionN

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# Save the trained model
checkpoint_path = path + "tft_model_mini.ckpt"
trainer.save_checkpoint(checkpoint_path)